<a href="https://colab.research.google.com/github/hbhb0311/LikelionStudy/blob/main/4.NLP/2.Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 원-핫인코딩

In [ ]:
!pip install konlpy

In [15]:
import numpy as np
import time
from konlpy.tag import Komoran
komoran = Komoran()

In [16]:
# 명사만 추출
text = '지금은 자연어 처리 공부 중입니다. 얼마나 빠를까?'
nouns = komoran.nouns(text)

In [17]:
# 단어 사전 구축 & 단어별 인덱스 부여
dics = {}
for word in nouns:
  if word not in dics:
    dics[word] = len(dics)
print(dics)

{'지금': 0, '자연어': 1, '처리': 2, '공부': 3, '중': 4}


In [18]:
# 원-핫 인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
# eye 함수 : 단위행렬 만들어줌, targets의 순서대로 단위 행렬 만들어줌
print(one_hot_targets)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [19]:
# 원-핫 인코딩 예제 2
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[[1,2,0]]
# eye 함수 : 단위행렬 만들어줌
print(one_hot_targets)

[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


### Word2Vec

In [20]:
from gensim.models import Word2Vec

In [21]:
# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
  with open(filename, 'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    # splitlines : \n, \r 등을 기준으로 나누는 것 -> 한 줄씩 분리
    # split : sep = \t 기준으로 나누는 것 -> 각 열별로 분리됨
    data = data[1:]
  return data

In [22]:
# 리뷰 파일 읽어오기
start = time.time()

print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('/content/drive/MyDrive/멋쟁이사자처럼/dataset/ratings.txt')
print(len(review_data)) # 데이터의 개수 확인
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)


# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
# 리뷰 데이터에서 1번째 열에만 후기가 있음(id, doc, label 순으로 데이터에 존재)
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)


# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences = docs, size = 200, window = 4, min_count = 2, sg = 1)
print('3) word2vec 모델 학습 완료: ', time.time() - start)

# sentences : 모델 학습에 필요한 문장 데이터
# size : 단어 임베딩 벡터의 차원
# hs : 0 (0이 아닌 경우 음수 샘플링 사용), 1(모델 학습에 softmax 사용)
# min_count : 단어 최소 빈도 수 제한(설정된 빈도 수 미만의 단어들은 학습 X)
# -> 즉, 1번 나타난 단어는 학습에 사용되지 않음
# sg : 0(CBOW 모델), 1(skip-gram 모델)


# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)


# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print('corpus_count: ', model.corpus_count)
print('corpus_total_words: ', model.corpus_total_words)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  2.280844211578369
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  134.75902485847473
3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  161.29787135124207
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  161.67508578300476
corpus_count:  200000
corpus_total_words:  1076896


#### 모델 불러와 임베딩 된 값 확인

In [23]:
# 모델 로딩
model = Word2Vec.load('nvmc.model')
print('사랑: ', model.wv['사랑'])
# 사랑이라는 단어로 생성한 단어 임베딩 벡터 -> 모델 학습시킬 때 사용한 사이즈로 벡터 차원이 결정됨

사랑:  [ 0.27141044  0.2153113   0.12079716 -0.15842655  0.06376249 -0.12702957
 -0.5127922  -0.32647645  0.3990108  -0.06863586  0.22982189  0.3128365
 -0.08811162 -0.15919013  0.14953505  0.39979106  0.263857    0.0041444
 -0.09068102  0.12556933 -0.272624   -0.19388725 -0.07759219 -0.01275868
  0.4883124   0.09408101 -0.03755792  0.03231707  0.27580473  0.00789384
  0.15037827 -0.03262882  0.09153937  0.43037686  0.26774812 -0.07836156
  0.23580582 -0.02477748  0.3745084  -0.146414    0.09628444  0.34085208
  0.3012133   0.4806978  -0.07595445  0.362257    0.18745215 -0.43127957
 -0.2939989  -0.17659777  0.00618889  0.6341174  -0.10249962 -0.3930558
  0.4546832   0.09840408  0.33111194 -0.0099607  -0.18999062  0.18079963
 -0.22904779 -0.08637014 -0.2661426   0.02426932  0.35550523 -0.5178979
 -0.04535771  0.26948908 -0.07015716 -0.23566675  0.05173555  0.09181086
 -0.4745562   0.2225199   0.04452609  0.4168042  -0.05302289  0.15629572
  0.01139535 -0.00863414  0.5770927  -0.43317083 -

In [24]:
# 단어 유사도 계산
model.wv.similarity('해리', '시리즈')

0.79732734

In [25]:
# 가장 유사한 단어 추출(벡터 공간에서 가장 가까운 거리에 있는 단어 반환)
model.wv.most_similar('시리즈', topn = 5)

[('엑스맨', 0.8446062803268433),
 ('포터', 0.8389245271682739),
 ('스타워즈', 0.8081360459327698),
 ('반지의 제왕', 0.8041760325431824),
 ('다이하드', 0.7987931966781616)]